In [ ]:
# Fijate si te parece bien esto y si ya no te explota la memoria.
# Traté de seguir tu línea pero no tocar nada de lo tuyo por las dudas
# Hay que tomar muchas decisiones todavía, pero creo que va encaminándose
# Cualquier cosa preguntame

In [ ]:
import glob
import pandas as pd
import json
import os
import nltk
from shutil import copyfile

In [ ]:
# Descomentar si no se tiene el recurso
#nltk.download("punkt")

In [ ]:
tags_mapping={
    "V":{
        "subcategoria":{
            "posicion": 1
        },
        "modo": {
            "posicion": 2,
            "I": "IND",
            "S": "SUBJ",
            "M": "IMP",
            "N": "INF",
            "G": "GDIO",
            "P":"PPIO"
        },
        "tiempo": {
            "posicion": 3,
            "P":"presente",
            "I": "imperfecto",
            "F": "futuro",
            "S": "pasado",
            "C": "condicion",
            "0": "atemp"
        },
        "persona":{
            "posicion": 4
        },
        "numero":{
            "posicion": 5,
            "S":"sg",
            "P": "pl"
        }
    },
    "N":{
        "subcategoria":{ # esta subcategoría no la vamos a usar en principio porque freeling 
            # no tiene nombres propios, y puse para agregarlos aparte con otra categoría
            # PropN, pero dejo esto acá por si eventualmente queremos usarlo.
            "posicion": 1,
            "C": "COMUN", 
            "P": "PROPIO"
        },
        "genero": {
            "posicion": 2,
            "M": "masc",
            "F": "fem",
            "C": "?gen"
        },
        "numero": {
            "posicion": 3,
            "S":"sg",
            "P": "pl",
            "N": "inv"
        }
    },
    "A":{
        "subcategoria":{
            "posicion": 1,
            "Q": "calif", 
            "O": "ord",
            "P": "pron"
        },
        "grado": {
            "posicion": 2,
            "A": "aum",
            "D": "dim",
            "C": "comp",
            "S": "superl"
        },
        "genero": {
            "posicion": 3,
            "M": "masc",
            "F": "fem",
            "C": "?gen"
        },
        "numero": {
            "posicion": 4,
            "S":"sg",
            "P": "pl",
            "N": "inv"
        },
        "funcion": { # This category denotes function in participles and adjectives but person in pronouns
            "posicion": 5,
            "0": "adj",
            "P": "part"
        }
    }
}

In [ ]:
class MakeRule:
    
    def __init__(self,word_dict):
        
        self.tag = word_dict.get("tag")
        self.lemma = word_dict.get("lema")
        self.shape = word_dict.get("forma")
        self.category = self.tag[0]
        self.mapping = tags_mapping.get(self.category)

        if self.mapping:

            for k in self.mapping.keys():
                setattr(self,k,self._get_mapped_tag(k))

            self.rule = self._make_rule()
            
        else:
            self.rule = ""
                
    def _get_mapped_tag(self,rasgo):
        rasgo_mapping = self.mapping.get(rasgo)
        position_tag = rasgo_mapping.get("posicion")
        return rasgo_mapping.get(self.tag[position_tag],self.tag[position_tag])
                
    def _make_rule(self):
        # Habría que agregar un elif por cada categoría a la que le definamos el mapeo de keys.
        # Los atributos son las keys del diccionario. category, shape y lemma siempre se llaman
        # así porque se definen antes (es para todas las clases de palabras igual)
        # Está así cortado para que sea más legible, pero podría estar todo en una línea
        if self.category == "V":
            rule = f"{self.category}[SUBCAT={self.subcategoria},MODE={self.modo},"\
                    f"TENSE={self.tiempo},PER={self.persona},"\
                    f"NUM={self.numero},SEM=<\e.({self.lemma}(e) "\
                    f"& {self.tiempo}(e))>] -> '{self.shape}'"
        if self.category == "N":
            rule = f"{self.category}[NUM={self.numero},GEN={self.genero},"\
                    f"SEM=<\\x.({self.lemma}(x))>] -> '{self.shape}'"
        if self.category == "A":
            if self.subcategoria != "pron":
                rule = f"{self.category}[NUM={self.numero},GEN={self.genero},"\
                    f"SUBCAT={self.subcategoria}, FUNCTION={self.funcion},"\
                    f"SEM=<\\x.({self.lemma}(x))>] -> '{self.shape}'"
            elif self.category == "A" and self.subcategoria == "pron":
                rule = f"{self.category}[NUM={self.numero},GEN={self.genero},"\
                    f"SUBCAT={self.subcategoria},"\
                    f"SEM=<\\x.(de-{self.funcion}-persona(x))>] -> '{self.shape}'"
        return rule          

In [ ]:
def load_vocabulary():
    freeling_all = list()
    vocab_df = pd.DataFrame()
    for freeling_file in glob.glob("./freeling/*.*.txt"):
        vocab_df = vocab_df.append(pd.read_csv(freeling_file, sep=" ",names=["forma","lema","tag"]))
    return vocab_df

def create_tmp_grammar():
    tmp_grammar = "tmp_grammar.fcfg"
    copyfile("../grammars/GramaticaDeRasgosBase.txt", tmp_grammar)
    return tmp_grammar
    

def append_rules_to_grammar(rules):
    with open("tmp_grammar.fcfg","a+") as current_grammar:
        for rule in rules:
             current_grammar.write(f"{rule}\n")


In [ ]:
def get_rules(word):
    word_rules = list()
    word_in_vocab = json.loads(vocab_df[vocab_df.forma == word].to_json(orient="records"))
    for entry in word_in_vocab:
        word_rules.append(MakeRule(entry).rule)
    return word_rules

In [ ]:
def interpret_sentence(sentence):
    tmp_grammar = create_tmp_grammar()
    tok_sentence = nltk.tokenize.word_tokenize(sentence)
    for word in set(tok_sentence):
        rules = get_rules(word)
        append_rules_to_grammar(rules)
    interpretation = nltk.interpret_sents([sentence], tmp_grammar)
    os.remove(tmp_grammar)
    return interpretation


In [ ]:
vocab_df = load_vocabulary()

In [ ]:
sentence = "vos repiqueteás"
interpretation = interpret_sentence(sentence)

In [ ]:
print(interpretation)

In [ ]:
for results in interpretation:
    for (synrep, semrep) in results:
        print(semrep)